In [1]:
import ee
import geemap
import geopandas as gpd
import os
from datetime import datetime, timedelta
import xarray as xr
import rioxarray
import rasterio
from rasterio.enums import Resampling
import numpy as np
from collections import defaultdict

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='btech-project-443813') # Make sure this project ID is correct

In [4]:
import ee
import geemap
import os
from datetime import datetime, timedelta

def initialize_ee():
    """Initialize Earth Engine"""
    try:
        ee.Initialize()
        print("Earth Engine initialized successfully")
    except Exception as e:
        print(f"Error initializing Earth Engine: {str(e)}")
        raise

def create_new_delhi_rectangle():
    """Create Earth Engine rectangle geometry for New Delhi region"""
    return ee.Geometry.Rectangle([
        76.8,  # min_lon (Western boundary)
        28.4,  # min_lat (Southern boundary)
        77.6,  # max_lon (Eastern boundary)
        28.9   # max_lat (Northern boundary)
    ])

def kelvin_to_celsius(image):
    """Convert LST from Kelvin to Celsius"""
    lst_band = image.select('LST_Day_1km').multiply(0.02)  # Scale factor applied
    lst_celsius = lst_band.subtract(273.15)  # Convert K to °C
    return image.addBands(lst_celsius.rename('LST_Celsius'))

def get_composite_lst(start_date, roi):
    """Get 8-day composite LST"""
    try:
        # Convert datetime to ee.Date
        ee_start_date = ee.Date(start_date.strftime('%Y-%m-%d'))
        ee_end_date = ee_start_date.advance(8, 'day')
        
        # Get MODIS LST 8-day composite data (MOD11A2.061 product)
        dataset = ee.ImageCollection('MODIS/061/MOD11A2') \
            .filterDate(ee_start_date, ee_end_date) \
            .filterBounds(roi)
        
        # Get LST image and apply quality filter
        lst_image = dataset.mean()  # Use mean to get composite
        
        # Check if we have data
        if dataset.size().getInfo() == 0:
            raise Exception("No LST data available for this period")
        
        # Get quality control band and cast to unsigned integer
        qc = lst_image.select('QC_Day').toUint8()
        
        # Create quality mask (bits 1-0 = 00, meaning good quality)
        # MODIS QC bits: 00=good quality, 01=other quality, 10=invalid, 11=cloud
        good_quality = qc.bitwiseAnd(ee.Number(3)).eq(0)
        
        # Apply quality mask and convert to Celsius
        lst_processed = kelvin_to_celsius(lst_image) \
            .updateMask(good_quality) \
            .select('LST_Celsius') \
            .clip(roi)
        
        return lst_processed
    
    except Exception as e:
        print(f"Error processing LST composite for {start_date.strftime('%Y-%m-%d')}: {str(e)}")
        raise

def download_composite_lst(start_date, roi, output_directory):
    """Download 8-day composite LST"""
    try:
        date_str = start_date.strftime('%Y-%m-%d')
        lst = get_composite_lst(start_date, roi)
        filename = os.path.join(output_directory, f'lst_delhi_composite_{date_str}.tif')
        
        # Get the region coordinates for geemap
        region = roi.getInfo()['coordinates'][0]
        
        # Use geemap to download and convert to GeoTIFF
        geemap.ee_export_image(
            lst,
            filename=filename,
            scale=1000,  # 1km resolution
            region=region,
            crs='EPSG:4326',
            file_per_band=False
        )
        
        if os.path.exists(filename):
            file_size = os.path.getsize(filename)
            print(f"Successfully downloaded LST composite for {date_str} (Size: {file_size/1024:.1f} KB)")
            return True
        else:
            print(f"Failed to download LST composite for {date_str}")
            return False
            
    except Exception as e:
        print(f"Error downloading LST composite for {date_str}: {str(e)}")
        return False

def main():
    # Initialize Earth Engine
    initialize_ee()
    
    # Define the region of interest for Southern India
    roi = create_new_delhi_rectangle()
    
    # Define the date range
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2023, 3, 31)
    
    # Create output directory
    output_directory = r"/home/stormej/dev/btech-project/data/lst_tif"
    os.makedirs(output_directory, exist_ok=True)
    
    # Download 8-day composite LST data for each period in the date range
    current_date = start_date
    successful_downloads = 0
    failed_downloads = 0
    
    print(f"\nStarting download of 8-day composite LST data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
    
    while current_date <= end_date:
        try:
            print(f"\nProcessing {current_date.strftime('%Y-%m-%d')} to {(current_date + timedelta(days=7)).strftime('%Y-%m-%d')}")
            
            if download_composite_lst(current_date, roi, output_directory):
                successful_downloads += 1
            else:
                failed_downloads += 1
                
        except Exception as e:
            print(f"Error processing {current_date.strftime('%Y-%m-%d')}: {str(e)}")
            failed_downloads += 1
            
        # Move to the next 8-day period
        current_date += timedelta(days=8)
    
    print("\nDownload Summary:")
    print(f"Successfully downloaded: {successful_downloads} periods")
    print(f"Failed downloads: {failed_downloads} periods")
    print("Process completed.")

if __name__ == "__main__":
    main()


Earth Engine initialized successfully

Starting download of 8-day composite LST data from 2000-01-01 to 2023-03-31...

Processing 2000-01-01 to 2000-01-08
Error processing LST composite for 2000-01-01: No LST data available for this period
Error downloading LST composite for 2000-01-01: No LST data available for this period

Processing 2000-01-09 to 2000-01-16
Error processing LST composite for 2000-01-09: No LST data available for this period
Error downloading LST composite for 2000-01-09: No LST data available for this period

Processing 2000-01-17 to 2000-01-24
Error processing LST composite for 2000-01-17: No LST data available for this period
Error downloading LST composite for 2000-01-17: No LST data available for this period

Processing 2000-01-25 to 2000-02-01
Error processing LST composite for 2000-01-25: No LST data available for this period
Error downloading LST composite for 2000-01-25: No LST data available for this period

Processing 2000-02-02 to 2000-02-09
Error process

In [ ]:
#Resampling lst monthly data to imd data i.e 0.25 degree

import os
import rasterio
from rasterio.enums import Resampling
# # Paths
# rainfall_folder = r'/home/stormej/dev/btech-project/data/rain_tif'  # Folder containing all rainfall files
rainfall_folder = r'/home/stormej/dev/btech-project/data/reference_tif'  # Folder containing reference files
lst_folder = r'/home/stormej/dev/btech-project/data/lst_tif_monthly'  # Folder containing all LST files
output_folder = r'/home/stormej/dev/btech-project/data/lst_composite_delhi_monthly_resampled'  # Folder to save resampled files

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process all LST files in the folder
for filename in os.listdir(lst_folder):
    if filename.endswith('.tif'):
        lst_path = os.path.join(lst_folder, filename)
        print(f"\nProcessing: {filename}")
        
        with rasterio.open(lst_path) as lst_src:
            lst_data = lst_src.read(1)
            print(f"LST data shape: {lst_data.shape}, dtype: {lst_data.dtype}")
            
            # Resample LST data using average method
            resampled_lst = lst_src.read(
                out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
                resampling=Resampling.average
            )
            print(f"Resampled LST shape: {resampled_lst.shape}, dtype: {resampled_lst.dtype}")
            
            # Extract date from LST filename (handling format: lst_south_2010-01-01.tif)
            # Split by underscore and get the last part before .tif
            date_part = filename.split('_')[-1].split('.')[0]
            
            # Create output filename
            output_filename = f'modis_lst_avg_resampled_{date_part}.tif'
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the resampled LST
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=resampled_lst.shape[1],
                width=resampled_lst.shape[2],
                count=1,
                dtype=resampled_lst.dtype,
                crs=rainfall_crs,
                transform=rainfall_transform,
            ) as dst:
                dst.write(resampled_lst[0], 1)
        
        print(f"Resampled LST saved to: {output_path}")

print("\nAll LST files have been resampled.")

Using reference rainfall file: /home/stormej/dev/btech-project/data/reference_tif/refrence_tif.tif
Reference rainfall data shape: (129, 135), CRS: EPSG:4326

Processing: lst_monthly_mean_2000-02.tif
LST data shape: (57, 90), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/lst_composite_delhi_monthly_resampled/modis_lst_avg_resampled_2000-02.tif

Processing: lst_monthly_mean_2000-03.tif
LST data shape: (57, 90), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/lst_composite_delhi_monthly_resampled/modis_lst_avg_resampled_2000-03.tif

Processing: lst_monthly_mean_2000-04.tif
LST data shape: (57, 90), dtype: float32
Resampled LST shape: (1, 129, 135), dtype: float32
Resampled LST saved to: /home/stormej/dev/btech-project/data/lst_composite_delhi_monthly_resampled/modis_lst_avg_resampled_2000-04.tif

Processing: lst_monthly_mean_2000-0